In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings(action = 'ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import keras
warnings.filterwarnings("ignore")
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense,Dropout
from pycaret.classification import *

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('data.csv')

In [3]:
df.head(5)

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,0,2000-01,27,18,NaN,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,0,2000-01,22,15,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,0,2000-01,45,16,1.0,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,0,2000-01,52,22,0.0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,0,2000-01,19,0,1.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5


In [4]:
df.shape

(30697, 25)

In [5]:
pd.DataFrame(data = {"featurs":df.dtypes.values,
                     "is null":df.isnull().sum().values},
             index = df.dtypes.index)

,featurs,is null
action_type,object,0
combined_shot_type,object,0
game_event_id,int64,0
game_id,int64,0
lat,float64,0
loc_x,int64,0
loc_y,int64,0
lon,float64,0
minutes_remaining,int64,0
period,int64,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30697 entries, 0 to 30696
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   action_type         30697 non-null  object 
 1   combined_shot_type  30697 non-null  object 
 2   game_event_id       30697 non-null  int64  
 3   game_id             30697 non-null  int64  
 4   lat                 30697 non-null  float64
 5   loc_x               30697 non-null  int64  
 6   loc_y               30697 non-null  int64  
 7   lon                 30697 non-null  float64
 8   minutes_remaining   30697 non-null  int64  
 9   period              30697 non-null  int64  
 10  playoffs            30697 non-null  int64  
 11  season              30697 non-null  object 
 12  seconds_remaining   30697 non-null  int64  
 13  shot_distance       30697 non-null  int64  
 14  shot_made_flag      25697 non-null  float64
 15  shot_type           30697 non-null  object 
 16  shot

In [7]:
Shots_to_predict = df[df['shot_made_flag'].isnull()]
Shots_to_train = df[df['shot_made_flag'].notna()]

In [8]:
abs(Shots_to_train.corr()['shot_made_flag']).sort_values(ascending=False)

shot_made_flag       1.000000
shot_distance        0.198242
loc_y                0.148070
lat                  0.148070
game_event_id        0.037232
period               0.032152
seconds_remaining    0.030804
minutes_remaining    0.028342
shot_id              0.013016
game_id              0.001612
playoffs             0.001257
lon                  0.000848
loc_x                0.000848
team_id                   NaN
Name: shot_made_flag, dtype: float64

In [9]:
df3=df.drop(df.columns[[0,3,5,7,10,11,15,16,17,18,19,20,21,22,23,24]], axis=1)  

In [10]:
df3=pd.get_dummies(df3,drop_first=True)

In [11]:
Shots_to_predict3 = df3[df3['shot_made_flag'].isnull()]
Shots_to_train3 = df3[df3['shot_made_flag'].notna()]

In [12]:
x,y=Shots_to_train3.drop('shot_made_flag',axis=1),Shots_to_train3[['shot_made_flag']]

In [13]:
model = Sequential()
model.add(Dense(81,activation = "relu"))
model.add(Dense(40,activation = "relu"))
model.add(Dense(27,activation = "relu"))
model.add(Dense(19,activation = "relu"))
model.add(Dense(5,activation = "relu"))
model.add(Dense(3,activation = "relu"))
model.add(Dense(1))

In [14]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])

In [15]:
history=model.fit(x,y,epochs=80,batch_size=12,verbose=1)

Epoch 1/80
2142/2142 [==============================] - 4s 2ms/step - loss: 0.9018 - accuracy: 0.5536
Epoch 2/80
2142/2142 [==============================] - 4s 2ms/step - loss: 0.6909 - accuracy: 0.5538
Epoch 3/80
2142/2142 [==============================] - 4s 2ms/step - loss: 0.6875 - accuracy: 0.5538
Epoch 4/80
2142/2142 [==============================] - 4s 2ms/step - loss: 0.6875 - accuracy: 0.5538
Epoch 5/80
2142/2142 [==============================] - 4s 2ms/step - loss: 0.6875 - accuracy: 0.5538
Epoch 6/80
2142/2142 [==============================] - 4s 2ms/step - loss: 0.6874 - accuracy: 0.5538
Epoch 7/80
2142/2142 [==============================] - 4s 2ms/step - loss: 0.6875 - accuracy: 0.5538
Epoch 8/80
2142/2142 [==============================] - 3s 2ms/step - loss: 0.6875 - accuracy: 0.5538
Epoch 9/80
2142/2142 [==============================] - 3s 2ms/step - loss: 0.6874 - accuracy: 0.5538
Epoch 10/80
2142/2142 [==============================] - 3s 2ms/step - loss: 0.687

2142/2142 [==============================] - 3s 2ms/step - loss: 0.6875 - accuracy: 0.5538


In [16]:
scores=model.evaluate(x,y)

804/804 [==============================] - 1s 1ms/step - loss: 0.6874 - accuracy: 0.5538


In [17]:
print("%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))

accuracy: 55.38%


In [18]:
model=Sequential()
model.add(Dense(20,input_dim=12,input_shape=(x.shape[1],),kernel_initializer='uniform',activation='relu'))
model.add(Dense(8,kernel_initializer='uniform',activation='relu'))
model.add(Dense(1,kernel_initializer='uniform',activation='sigmoid'))

In [19]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
model.fit(x,y,epochs=150,batch_size=10,verbose=1)

Epoch 1/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6709 - accuracy: 0.5905
Epoch 2/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6620 - accuracy: 0.6066
Epoch 3/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6569 - accuracy: 0.6068
Epoch 4/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6554 - accuracy: 0.6072
Epoch 5/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6544 - accuracy: 0.6083
Epoch 6/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6546 - accuracy: 0.6085
Epoch 7/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6541 - accuracy: 0.6063
Epoch 8/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6533 - accuracy: 0.6083
Epoch 9/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6535 - accuracy: 0.6100
Epoch 10/150
2570/2570 [==============================] - 3s 1ms/step - l

2570/2570 [==============================] - 3s 1ms/step - loss: 0.6494 - accuracy: 0.6152
Epoch 80/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6497 - accuracy: 0.6129
Epoch 81/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6493 - accuracy: 0.6150
Epoch 82/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6489 - accuracy: 0.6146
Epoch 83/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6493 - accuracy: 0.6149
Epoch 84/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6496 - accuracy: 0.6146
Epoch 85/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6489 - accuracy: 0.6152
Epoch 86/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6495 - accuracy: 0.6145
Epoch 87/150
2570/2570 [==============================] - 3s 1ms/step - loss: 0.6494 - accuracy: 0.6146
Epoch 88/150
2570/2570 [==============================] - 3s 1ms/step - loss:

In [21]:
scores=model.evaluate(x,y)

804/804 [==============================] - 1s 955us/step - loss: 0.6471 - accuracy: 0.6161


In [22]:
print("%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))

accuracy: 61.61%


In [23]:
Shots_to_predict3.drop('shot_made_flag',axis=1,inplace=True)

In [24]:
y_pred=model.predict(Shots_to_predict3)

157/157 [==============================] - 0s 901us/step


In [25]:
y_pred.dtype

dtype('float32')

In [26]:
y_pred=y_pred.round()
y_pred

array([[0.],
       [0.],
       [1.],
       ...,
       [0.],
       [1.],
       [0.]], dtype=float32)

In [27]:
def write_prediction(prediction, name):
    id = np.array(Shots_to_predict['shot_id']).astype(int)
    solution = pd.DataFrame(prediction, id, columns = ['shot_made_flag'])
    solution.to_csv(name, index_label = ['shot_id'])

In [28]:
write_prediction(y_pred, "My_output.csv")